In [4]:
import os 



In [5]:
%pwd

'/media/puzan/NewVolume/MyFolder'

In [6]:
# os.chdir("../")

In [7]:
os.getcwd()

'/media/puzan/NewVolume/MyFolder'

In [8]:
#create entity that is return type
from dataclasses import dataclass
from pathlib import Path
#data class bhaneko class banauxa instance haru

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path  # Correctly typed and declared


In [9]:
%pwd

'/media/puzan/NewVolume/MyFolder'

In [10]:
from translator.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from translator.utils.common import read_yaml,create_directories,get_size


In [11]:
print(CONFIG_FILE_PATH,PARAMS_FILE_PATH)

config/config.yaml param/params.yaml


In [12]:
with open(CONFIG_FILE_PATH,"r") as f:
    print(f.readlines())

['artifacts_root: artifacts\n', '\n', 'data_ingestion:\n', '  root_dir: artifacts/data_ingestion\n', '  source_URL: https://github.com/Puzan789/Datas/blob/main/translatordata.zip\n', '  local_data_file: artifacts/data_ingestion/data.zip\n', '  unzip_dir: artifacts/data_ingestion\n']


In [13]:
# configuration manager

In [24]:
# class ConfigurationManager:
#     def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
#         self.config=read_yaml(config_file_path)
#         self.params=read_yaml(params_file_path)
#         create_directories([self.config.artifacts_root])
    
#     def get_data_ingestion_config(self) -> DataIngestionConfig:
#         config=self.config.data_ingestion
#         create_directories([Path(config.root_dir)])
#         data_ingestion_config=DataIngestionConfig(
#             root_dir=Path(config.root_dir),
#             source_URL=config.source_URL,
#             local_data_file=config.local_data_file,
#             unzip_dir=config.unzip_dir
#         )
#         return data_ingestion_config

class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([Path(config.root_dir)])  # Ensure it's a Path object
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),  # Convert to Path object
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),  # Convert to Path object
            unzip_dir=Path(config.unzip_dir),  # Convert to Path object
        )
        return data_ingestion_config

    

In [23]:
#components 
import os 
import urllib.request as request
import zipfile
from translator.logging import logger
from translator.utils import common


In [31]:
class Dataingestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not self.config.local_data_file.exists() or os.path.getsize(self.config.local_data_file)==0:
            try:
                file_name,header=request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{file_name} downloaded with the following {header}")
                #verifying whether the file is empty or not
                if os.path.getsize(self.config.local_data_file)==0:
                    logger.error("Download file is empty.")
                    raise Exception ("Empty downloaded file")
            except Exception as e :
                raise f"Error occured while downloading the file {e}"
        else:
            logger.info (f"file already exists with the size {get_size(self.config.local_data_file)}")
    
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        logger.info(f"Extracting zip file to {unzip_path}")
        try :
            with zipfile.ZipFile(self.config.local_data_file,'r') as zip_f:
                zip_f.extractall(unzip_path)
                logger.info("Extraction completed successfully")
        except zipfile.BadZipFile:
            logger.error(f"Bad zip file {self.config.local_data_file}")
        except zipfile.LargeZipFile:
            logger.error(f"Zip file too large{self.config.local_data_file}")
        except Exception as e:
            logger.error(f"Error occured while extracting zip file {e}")
            raise
            
        
    



In [32]:
#pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    Dataingestion=Dataingestion(data_ingestion_config)
    Dataingestion.download_file()
    Dataingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occured while")
    raise e
    

[2024-09-25 08:58:55,634:INFO:common:yaml_file:config/config.yaml loaded successfully
[2024-09-25 08:58:55,639:INFO:common:yaml_file:param/params.yaml loaded successfully
[2024-09-25 08:58:55,641:INFO:common:Directory 'artifacts' created successfully
[2024-09-25 08:58:55,642:INFO:common:Directory 'artifacts/data_ingestion' created successfully
[2024-09-25 08:58:56,784:INFO:2124288436:artifacts/data_ingestion/data.zip downloaded with the following Connection: close
Content-Length: 5349
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "ae983b6b1b4db56278ef2af0f64aca08a49ee1262cca45c79571c6d2e840d41f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6C9E:175DB:369A39:44304A:66F37FF3
Accept-Ranges: bytes
Date: Wed, 25 Sep 2024 03:13:56 GMT
Via: 1.1 varnish
X-Served-By: cache-del21733-DEL
X-Cache: 